In [1]:
# Import scraping modules
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Import data manipulation modules
import pandas as pd
import numpy as np

# Import data visualization modules
import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly.graph_objects as go

# Passing Comparisons

In [43]:
# URL of page
season = 2020

url = f'https://www.pro-football-reference.com/years/{season}/passing.htm'
# Open URL and pass to BeautifulSoup
html = urlopen(url)
stats_page = BeautifulSoup(html)

# Collect table headers
column_headers = stats_page.findAll('tr')[0]
column_headers = [i.getText() for i in column_headers.findAll('th')]

# Collect table rows
rows = stats_page.findAll('tr')[1:]

# Get stats from each row
qb_stats = []
for i in range(len(rows)):
    qb_stats.append([col.getText() for col in rows[i].findAll('td')])
    
# Create DataFrame from our scraped data
data = pd.DataFrame(qb_stats, columns=column_headers[1:])
data.head()
data['Season'] = season

# Rename sack yards column to `Yds_Sack`
new_columns = data.columns.values
new_columns[-7] = 'Yds_Sack'
data.columns = new_columns

data.head(20)

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Rate,QBR,Sk,Yds_Sack,NY/A,ANY/A,Sk%,4QC,GWD,Season
0,Matt Ryan,ATL,35,QB,16,16,4-12-0,407,626,65.0,...,93.3,66.9,41,257,6.48,6.52,6.1,,,2020
1,Tom Brady,TAM,43,QB,16,16,11-5-0,401,610,65.7,...,102.2,72.6,21,143,7.12,7.53,3.3,3,3,2020
2,Ben Roethlisberger,PIT,38,QB,15,15,12-3-0,399,608,65.6,...,94.1,60.1,13,118,5.93,6.27,2.1,4,4,2020
3,Justin Herbert,LAC,22,QB,15,15,6-9-0,396,595,66.6,...,98.3,69.6,32,218,6.57,6.84,5.1,1,3,2020
4,Patrick Mahomes *,KAN,25,QB,15,15,14-1-0,390,588,66.3,...,108.2,82.9,22,147,7.53,8.33,3.6,3,3,2020
5,Josh Allen *,BUF,24,QB,16,16,13-3-0,396,572,69.2,...,107.2,81.7,26,159,7.33,7.82,4.3,2,3,2020
6,Kyler Murray *,ARI,23,QB,16,16,8-8-0,375,558,67.2,...,94.3,68.9,27,176,6.49,6.45,4.6,3,4,2020
7,Russell Wilson *,SEA,32,QB,16,16,12-4-0,384,558,68.8,...,105.1,73.5,47,301,6.46,6.82,7.8,3,3,2020
8,Jared Goff,LAR,26,QB,15,15,9-6-0,370,552,67.0,...,90.0,58.4,23,161,6.59,6.27,4.0,0,1,2020
9,Deshaun Watson *,HOU,25,QB,16,16,4-12-0,382,544,70.2,...,112.4,70.5,49,293,7.64,8.22,8.3,,,2020


In [44]:
player1 = 'Russell Wilson'
player2 = 'Drew Lock'
target_season = 2020

In [45]:
#filter by target season
data_radar = data.loc[data['Season'] == target_season]

# Select stat categories
categories = ['Cmp%', 'Yds', 'TD', 'Int', 'Y/A', 'Rate']

# Create data subset for radar chart
data_radar = data[['Player', 'Tm'] + categories]

# Convert data to numerical values
for i in categories:
    data_radar[i] = pd.to_numeric(data[i])
    
# Remove ornamental characters for achievements
data_radar['Player'] = data_radar['Player'].str.replace('*', '')
data_radar['Player'] = data_radar['Player'].str.replace('+', '')
data_radar['Player'] = data_radar['Player'].str.strip()

# Filter by passing yards
data_radar = data_radar[data_radar['Yds'] > 1500]

# Create columns with percentile rank
for i in categories:
    data_radar[i + '_Rank'] = data_radar[i].rank(pct=True)

# We need to flip the rank for interceptions
#data_radar['Int_Rank'] = 1 - data_radar['Int_Rank']

# Examine data
data_radar.head()



<ipython-input-45-3d481939b54e>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-3d481939b54e>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-3d481939b54e>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-45-3d48

,Player,Tm,Cmp%,Yds,TD,Int,Y/A,Rate,Cmp%_Rank,Yds_Rank,TD_Rank,Int_Rank,Y/A_Rank,Rate_Rank
0,Matt Ryan,ATL,65.0,4581.0,26.0,11.0,7.3,93.3,0.342857,0.914286,0.628571,0.728571,0.500000,0.400000
1,Tom Brady,TAM,65.7,4633.0,40.0,12.0,7.6,102.2,0.457143,0.942857,0.957143,0.828571,0.671429,0.771429
2,Ben Roethlisberger,PIT,65.6,3803.0,33.0,10.0,6.3,94.1,0.428571,0.571429,0.800000,0.585714,0.142857,0.457143
3,Justin Herbert,LAC,66.6,4336.0,31.0,10.0,7.3,98.3,0.571429,0.857143,0.742857,0.585714,0.500000,0.657143
4,Patrick Mahomes,KAN,66.3,4740.0,38.0,6.0,8.1,108.2,0.542857,0.971429,0.914286,0.185714,0.885714,0.942857


In [46]:
list1 = data_radar.loc[data_radar['Player'] == player1, ['Cmp%_Rank','Yds_Rank','TD_Rank','Int_Rank','Y/A_Rank','Rate_Rank']].values
print(list1)

list2 = data_radar.loc[data_radar['Player'] == player2, ['Cmp%_Rank','Yds_Rank','TD_Rank','Int_Rank','Y/A_Rank','Rate_Rank']].values
print(list2)

[[0.85714286 0.77142857 0.95714286 0.91428571 0.6        0.82857143]]
[[0.02857143 0.42857143 0.41428571 0.98571429 0.24285714 0.08571429]]


In [47]:
#plot
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=list1[0],
      theta=categories,
      fill='toself',
      name=player1
))
fig.add_trace(go.Scatterpolar(
      r=list2[0],
      theta=categories,
      fill='toself',
      name=player2
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=False,
      range=[0, 1]
    )),
  showlegend=True
)

fig.show()